# 1. Data collection

## 1.1 Get the list of animes

In [1]:
from bs4 import BeautifulSoup as bs
import requests

In [2]:
def getUrls(start, stop): 

    urls = []

    for i in range(start, stop):
        url = 'https://myanimelist.net/topanime.php?limit='+str(i*50)
        r = requests.get(url)
        html_content = r.text
        soup = bs(html_content, 'lxml')
        links = soup.find_all('h3') 

        for anime in links[:-3]:
            if anime.find('a'):
                urls.append(anime.find('a')['href'])
    return urls

In [3]:
### PROVAAAAAA

f = open("topAnime.txt", 'w', encoding="utf8")
f.write('\n'.join(getUrls(0,20)))
f.close

<function TextIOWrapper.close()>

## 1.2 Crawl animes

In [7]:
import os

def saveHtml(page):
#saving the HTML file of page 'page' in the corresponding folder

    subfolder = "downloaded_Html/page_{}".format(page)
    os.makedirs(subfolder)

    f = open("topAnime.txt", 'r', encoding="utf8")
    lines = f.readlines()[(page-1)*50:(page)*50]
    lines = [line.rstrip() for line in lines]
    f.close

    i = 1+50*(page-1)
    for link in lines:
        html = requests.get(link)
        file_name = '{}/{}.html'.format(subfolder, i)
        g = open(file_name, 'w', encoding="utf8")
        g.write(html.text)
        g.close
        i += 1

In [9]:
#### PROVAAAAA

saveHtml(3)

## 1.3 Parse downloaded pages